In [ ]:
# Start with loading all necessary libraries

import spacy
import os
import csv
from itertools import zip_longest
import nltk
import wordcloud
import json
from spacy.lang.en.stop_words import STOP_WORDS as en_stopwords
from spacy.lang.pt.stop_words import STOP_WORDS as pt_stopwords
from spacy.lang.es.stop_words import STOP_WORDS as es_stopwords
import sys
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
def get_stopwords(lang):
    if lang == 'english':
        df_stopwords = set(en_stopwords)
        custom_stopwords = set(['olympics', 'olympic', 'london', 'person',
                                'rio', 'legacy', '2012', '2016',
                                'said', 'caption', 'image', 'years',
                                'd','s','t','m','n','ve', 'll', 'xe2', 'x80', 'x99', 'x99s', 'nthe', 'lda', '000', 'xc2'])
        return df_stopwords | custom_stopwords
    elif lang == 'portuguese':
        df_stopwords = set(pt_stopwords)
        custom_stopwords = set(['legado', 'olimpico', 'london',
                                'olímpica', 'londres', 'rio', 'legado',
                                'olímpico', '2012', '2016'])
        return df_stopwords | custom_stopwords
    
    
def my_tokenizers(doc):
    words = nltk.word_tokenize(doc)
    return [word for word in words if len(word) > 2]

def get_top_n_words(corpus, stopwords, n=20):
    vec = CountVectorizer(stop_words = stopwords).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def get_top_n_bigram(corpus, stopwords, n=20):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words = stopwords).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def get_top_n_words_tfidf(corpus, stopwords, n=20):
    vec = TfidfVectorizer(stop_words = stopwords).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def get_top_n_bigram_tfidf(corpus, stopwords, n=20):
    vec = TfidfVectorizer(ngram_range=(2, 2), stop_words = stopwords).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
domain = 'bbc' #Try also 'bbc','businessinsider','buzzfeed','citymonitor','gamesmonitor','guardian','independentgoogle','itv','metro','queenelizabetholympicpark','sky','sun','theconversation','theindependent','uksport','g-bbc','g-dailymail','g-telegraph','g-guardian'
lang = 'english'
entitieslocal = './data/'+ domain + '/data_analysis/all_texts_together' + domain + '.txt' 
entitieslocal2 = './data/'+ domain + '/data_analysis/' 
nlp = spacy.load("en_core_web_sm")

In [ ]:
with open(os.path.join(entitieslocal), 'r', encoding='utf-8') as myfile: 
            documento = myfile.read()
            doc = nlp(documento)
                   

# document level
ents = [(e.label_, e.text) for e in doc.ents if not e.text.isspace()]
print(ents)  

In [ ]:
#------------ WRITE CSV WITH DATA ----------------#

entity = [(e.label_) for e in doc.ents if not e.text.isspace()]
word = [(e.text) for e in doc.ents if not e.text.isspace()]

d = [entity,word]
export_data = zip_longest(*d, fillvalue = '')
with open(entitieslocal2 + 'entexcel.csv', 'w', newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("Entity", "Text"))
      wr.writerows(export_data)
myfile.close()

In [ ]:
#------------ WRITE results in TXT ----------------#

Keyword = 'PERSON' # Try also PERSON, FAC, GPE, LOC, MONEY, ORG
list = []
with open(entitieslocal2 + 'entexcel.csv', 'r') as f:
    for row in csv.reader(f):
        try:
            if Keyword in row:
                list.append(row)
                #print(row)
                
                #writer.writerow(row)
        except:
            continue

In [ ]:
with open(os.path.join(entitieslocal2 + Keyword + '.txt'), 'w+') as file1:
    file1.write(str(list))

In [ ]:
#------------ Analyse results with NLTK ----------------#

with open(os.path.join(entitieslocal2 + Keyword + '.txt'), 'r') as file2: 
            docENT = file2.read()

In [ ]:
#TOP N WORDS FOR ENTITIES
get_top_n_words([docENT], get_stopwords(lang), 50)

In [ ]:
get_top_n_bigram([docENT], get_stopwords(lang), 30)

In [ ]:
#------------ WORDCLOUD ----------------#

text = docENT
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(['olympics', 'PERSON', """PERSON'""", """Olympic'"""])


In [ ]:
wordcloud = WordCloud(stopwords=stopwords, max_words=100, background_color="white", width=1500, height=800).generate(text)
#plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Save the image in the img folder:
wordcloud.to_file('./data_analysis/' + domain + Keyword +'.png')

In [ ]:
#------------------------------------ ENTIRE DATA ANALYSIS-----------------------------------------------------------#

In [ ]:
directory1 = 'bbc'
directory2 = 'businessinsider'
directory3 = 'buzzfeed'
directory4 = 'citymonitor'
directory5 = 'gamesmonitor'
directory6 = 'guardian'
directory7 = 'independentgoogle'
directory8 = 'itv'
directory9 = 'metro'
directory10 = 'queenelizabetholympicpark'
directory11 = 'sky'
directory12 = 'sun'
directory13 = 'theconversation'
directory14 = 'theindependent'
directory15 = 'uksport'
directory16 = 'g-bbc'
directory17 = 'g-dailymail'
directory18 = 'g-telegraph'
directory19 = 'g-guardian'


localolympic1 = './data/' + directory1 + '/data_analysis/'
localolympic2 = './data/' + directory2 + '/data_analysis/'
localolympic3 = './data/' + directory3 + '/data_analysis/'
localolympic4 = './data/' + directory4 + '/data_analysis/'
localolympic5 = './data/' + directory5 + '/data_analysis/'
localolympic6 = './data/' + directory6 + '/data_analysis/'
localolympic7 = './data/' + directory7 + '/data_analysis/'
localolympic8 = './data/' + directory8 + '/data_analysis/'
localolympic9 = './data/' + directory9 + '/data_analysis/'
localolympic10 = './data/' + directory10 + '/data_analysis/'
localolympic11 = './data/' + directory11 + '/data_analysis/'
localolympic12 = './data/' + directory12 + '/data_analysis/'
localolympic13 = './data/' + directory13 + '/data_analysis/'
localolympic14 = './data/' + directory14 + '/data_analysis/'
localolympic15 = './data/' + directory15 + '/data_analysis/'
localolympic16 = './data/' + directory16 + '/data_analysis/'
localolympic17 = './data/' + directory17 + '/data_analysis/'
localolympic18 = './data/' + directory18 + '/data_analysis/'
localolympic19 = './data/' + directory19 + '/data_analysis/'

In [ ]:
with open(os.path.join(localolympic1 + 'all_texts_together' + directory1 + '.txt'), 'r', encoding= 'utf8') as myfile1:
    content1 = myfile1.read()
with open(os.path.join(localolympic2 + 'all_texts_together' + directory2 + '.txt'), 'r', encoding= 'utf8') as myfile2:
    content2 = myfile2.read()
with open(os.path.join(localolympic3 + 'all_texts_together' + directory3 + '.txt'), 'r', encoding= 'utf8') as myfile3:
    content3 = myfile3.read()
with open(os.path.join(localolympic4 + 'all_texts_together' + directory4 + '.txt'), 'r', encoding= 'utf8') as myfile4:
    content4 = myfile4.read()
with open(os.path.join(localolympic5 + 'all_texts_together' + directory5 + '.txt'), 'r', encoding= 'utf8') as myfile5:
    content5 = myfile5.read()
with open(os.path.join(localolympic6 + 'all_texts_together' + directory6 + '.txt'), 'r', encoding= 'utf8') as myfile6:
    content6 = myfile6.read()
with open(os.path.join(localolympic7 + 'all_texts_together' + directory7 + '.txt'), 'r', encoding= 'utf8') as myfile7:
    content7 = myfile7.read()
with open(os.path.join(localolympic8 + 'all_texts_together' + directory8 + '.txt'), 'r', encoding= 'utf8') as myfile8:
    content8 = myfile8.read()
with open(os.path.join(localolympic9 + 'all_texts_together' + directory9 + '.txt'), 'r', encoding= 'utf8') as myfile9:
    content9 = myfile9.read()
with open(os.path.join(localolympic10 + 'all_texts_together' + directory10 + '.txt'), 'r', encoding= 'utf8') as myfile10:
    content10 = myfile10.read()
with open(os.path.join(localolympic11 + 'all_texts_together' + directory11 + '.txt'), 'r', encoding= 'utf8') as myfile11:
    content11 = myfile11.read()
with open(os.path.join(localolympic12 + 'all_texts_together' + directory12 + '.txt'), 'r', encoding= 'utf8') as myfile12:
    content12 = myfile12.read()
with open(os.path.join(localolympic13 + 'all_texts_together' + directory13 + '.txt'), 'r', encoding= 'utf8') as myfile13:
    content13 = myfile13.read()
with open(os.path.join(localolympic14 + 'all_texts_together' + directory14 + '.txt'), 'r', encoding= 'utf8') as myfile14:
    content14 = myfile14.read()
with open(os.path.join(localolympic15 + 'all_texts_together' + directory15 + '.txt'), 'r', encoding= 'utf8') as myfile15:
    content15 = myfile15.read()
with open(os.path.join(localolympic16 + 'all_texts_together' + directory16 + '.txt'), 'r', encoding= 'utf8') as myfile16:
    content16 = myfile16.read()
with open(os.path.join(localolympic17 + 'all_texts_together' + directory17 + '.txt'), 'r', encoding= 'utf8') as myfile17:
    content17 = myfile17.read()
with open(os.path.join(localolympic18 + 'all_texts_together' + directory18 + '.txt'), 'r', encoding= 'utf8') as myfile18:
    content18 = myfile18.read()
with open(os.path.join(localolympic19 + 'all_texts_together' + directory19 + '.txt'), 'r', encoding= 'utf8') as myfile19:
    content19 = myfile19.read()

In [ ]:
nlp.max_length = 11000000 
doc2 = nlp(content1+content2+content3+content4+content5+content6+content7+content8+content9+content10+content11+content12+content13+content14+content15+content16+content17+content18+content19)